### Get IPS time series data from mwa-solar onto Safs Local Machine

In [28]:
import numpy as np
from numpy.random import default_rng
import os
import math
from astropy.table import Table, conf, Column
import matplotlib.pyplot as plt
import h5py

from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

#Data
from imstack import ImageStack

In [29]:
# Read in Data
table = Table.read('bright_cat_pf.fits', format='fits')


def find_ts(table_row, imstack):
    # Returns the time series for a single observation. Table should be the single row for that observation 

    ra = table_row['ra_var'] 
    dec = table_row['dec_var']

    x, y = imstack.world2pix(ra, dec)
    time_series = imstack.pix2ts(x, y, correct=False)

    return time_series 



def find_offsource_ts(table_row, imstack):
    # Returns the time series for a single observation. Table should be the single row for that observation 

    ra = table_row['ra_var'] 
    dec = table_row['dec_var']

    x, y = imstack.world2pix(ra, dec)
    x = x +5 #pixels?
    y = y +5 #pixels>
    offsource_time_series = imstack.pix2ts(x, y, correct=False)

    return offsource_time_series 

In [30]:
table 

GLEAM,RAJ2000,DEJ2000,Fp162,elongation2,limb,mpt1,mpt2,pbcor,pbcor_norm,snr,snr_2,snr_scint,obsid,snr_cont,nsi1,nsi2,bright_name,nsi_fit,s_162,subset,source_elevation,source_azimuth,obs_elevation,snr_filt,elevation_filt,dir,uuid_2,ra_var,dec_var,moment2_background,peak_flux
,,,,,,,,,,,,,,,,,,,,,deg,deg,deg,,,,,,,,
bytes14,float32,float32,float32,float32,bytes1,float32,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32,bytes28,float32,float32,bytes5,float32,float32,float32,bool,bool,bytes50,bytes36,float32,float32,float32,float32
J000106-174125,0.27705,-17.690334,1.2137686,40.05951,E,0.22613288,0.20075312,0.4211798,0.46549094,38.362244,14.839206,14.183964,1233384720,223.52196,0.6738726,0.75906545,GLEAM J000106-174125,0.84610957,1.2137686,2019E,79.43271,31.576632,80.348,False,False,/data/ips/dr1p1,e4d79f98-919c-4d51-9106-c0a44d943ba8,0.27397075,-17.691341,0.07250194,0.51282907
J000106-174125,0.27705,-17.690334,1.2137686,39.131214,E,0.23333263,0.20566975,0.47218856,0.54634964,42.016983,20.771927,21.707634,1233472808,267.92947,0.6655634,0.7550827,GLEAM J000106-174125,0.84610957,1.2137686,2019E,80.64918,346.64407,74.6271,False,False,/data/ips/dr1p1,b3efd872-095c-4b06-a047-9a857f3492c7,0.27173635,-17.691729,0.07023699,0.60092515
J000106-174125,0.27705,-17.690334,1.2137686,38.225433,E,0.24082333,0.21070458,0.5443091,0.61138284,51.654804,22.462378,22.111752,1233559208,282.8307,0.64067906,0.7322597,GLEAM J000106-174125,0.84610957,1.2137686,2019E,80.40455,341.1296,74.6271,False,False,/data/ips/dr1p1,cdf46e7f-a722-4590-952c-b0811146c2aa,0.27451184,-17.68874,0.069683835,0.63318574
J000106-174125,0.27705,-17.690334,1.2137686,37.324665,E,0.24876857,0.2159569,0.58938724,0.6747931,44.454033,21.062778,19.958677,1233645608,292.24948,0.58298266,0.6715588,GLEAM J000106-174125,0.84610957,1.2137686,2019E,80.081894,335.9199,74.6271,False,False,/data/ips/dr1p1,29bfbf92-7149-4f42-9b98-e6ddcba8ffe3,0.27217227,-17.692554,0.06868792,0.62557113
J000106-174125,0.27705,-17.690334,1.2137686,33.80559,E,0.28545693,0.23908368,0.32057446,0.36178568,32.27803,9.549107,9.710536,1233988640,157.43408,0.59529257,0.71075696,GLEAM J000106-174125,0.84610957,1.2137686,2019E,80.58283,15.075764,67.9813,False,False,/data/ips/dr1p2,de3be8e8-60da-40ed-ae20-cb8f4b076066,0.27259687,-17.692059,0.07427959,0.35386178
J000106-174125,0.27705,-17.690334,1.2137686,33.783356,E,0.2857219,0.23924421,0.7462893,0.81491745,69.9226,42.013725,41.05206,1233990840,407.475,0.5745882,0.6862128,GLEAM J000106-174125,0.84610957,1.2137686,2019E,79.013504,324.70648,74.6271,False,False,/data/ips/dr1p1,78cabda7-f59e-4ca9-a415-fb3d6dfb98b0,0.27109703,-17.690195,0.07096795,0.90061134
J000106-174125,0.27705,-17.690334,1.2137686,26.298372,E,0.41097295,0.30605325,0.7178301,0.8544347,57.3414,87.89223,85.05073,1234768248,389.04248,0.7227177,0.97047627,GLEAM J000106-174125,0.84610957,1.2137686,2019E,73.63596,300.8317,70.4075,False,False,/data/ips/dr1p2,505f1fd9-4df6-482f-af51-b663ad3cf5d2,0.27165732,-17.689547,0.07022636,0.8460366
J000106-174125,0.27705,-17.690334,1.2137686,24.748901,E,0.44996828,0.3236476,0.45256546,0.558998,49.60462,32.618484,30.508043,1234943552,200.0701,0.6303456,0.8763715,GLEAM J000106-174125,0.84610957,1.2137686,2019E,63.334553,284.35602,64.6934,False,False,/data/ips/dr1p2,6592bf3d-45bd-4aa9-9a8b-769f91290385,0.2700725,-17.688667,0.08726336,0.5240144


In [ ]:
# table = table[:2]

In [31]:
N=len(table)
M=1152

on_source = np.zeros((N, M))
off_source = np.zeros((N, M))
is_valid = np.zeros((N,), dtype=(bool))

for i, row in tqdm(enumerate(table)):
    try:
        obsid = row['obsid']
        #Drill image cube for 600mS of data per observation
        source_path = os.path.join(row['dir'], str(row['obsid'])+".hdf5")
        source_imstack = ImageStack(source_path, freq="121-132")
        on_source[i] = find_ts(row, source_imstack)
        off_source[i] = find_offsource_ts(row, source_imstack)
        is_valid[i] = True

    except Exception as error:
        print(f'Error procession ObsID {obsid}: {error}')
        is_valid[i] = False
        continue
    

with h5py.File("timeseries_full.hdf5", "w") as f:
    h5_on_source = f.create_dataset("on_source", (N, M), chunks=(1, M), dtype=np.float16, compression='lzf', shuffle=True)
    h5_off_source = f.create_dataset("off_source", (N, M), chunks=(1, M), dtype=np.float16, compression='lzf', shuffle=True)
    h5_isvalid = f.create_dataset("is_valid", (N,), dtype=(bool))
    h5_on_source[...] = on_source
    h5_off_source[...] = off_source

19680it [40:47, 17.25it/s]

Error procession ObsID 1233367016: [Errno 13] Unable to synchronously open file (unable to open file: name = '/data/ips/galcen/1233367016/1233367016.hdf5', errno = 13, error message = 'Permission denied', flags = 0, o_flags = 0)
Error procession ObsID 1233369888: [Errno 13] Unable to synchronously open file (unable to open file: name = '/data/ips/galcen/1233369888/1233369888.hdf5', errno = 13, error message = 'Permission denied', flags = 0, o_flags = 0)
Error procession ObsID 1233538144: [Errno 13] Unable to synchronously open file (unable to open file: name = '/data/ips/galcen/1233538144/1233538144.hdf5', errno = 13, error message = 'Permission denied', flags = 0, o_flags = 0)
Error procession ObsID 1233542696: [Errno 13] Unable to synchronously open file (unable to open file: name = '/data/ips/galcen/1233542696/1233542696.hdf5', errno = 13, error message = 'Permission denied', flags = 0, o_flags = 0)
Error procession ObsID 1233624552: [Errno 13] Unable to synchronously open file (una

19694it [40:47, 28.62it/s]

Error procession ObsID 1233367016: [Errno 13] Unable to synchronously open file (unable to open file: name = '/data/ips/galcen/1233367016/1233367016.hdf5', errno = 13, error message = 'Permission denied', flags = 0, o_flags = 0)
Error procession ObsID 1233453896: [Errno 13] Unable to synchronously open file (unable to open file: name = '/data/ips/galcen/1233453896/1233453896.hdf5', errno = 13, error message = 'Permission denied', flags = 0, o_flags = 0)
Error procession ObsID 1233538144: [Errno 13] Unable to synchronously open file (unable to open file: name = '/data/ips/galcen/1233538144/1233538144.hdf5', errno = 13, error message = 'Permission denied', flags = 0, o_flags = 0)
Error procession ObsID 1233540304: [Errno 13] Unable to synchronously open file (unable to open file: name = '/data/ips/galcen/1233540304/1233540304.hdf5', errno = 13, error message = 'Permission denied', flags = 0, o_flags = 0)
Error procession ObsID 1233624552: [Errno 13] Unable to synchronously open file (una

19702it [40:47, 25.86it/s]

...

22370it [44:12, 12.25it/s]

Error procession ObsID 1238114824: could not broadcast input array from shape (1107,) into shape (1152,)


22390it [44:13, 15.36it/s]

Error procession ObsID 1238114824: could not broadcast input array from shape (1107,) into shape (1152,)


22573it [44:30,  8.45it/s]
